# 🔬 Sofia Pulse - Data Mining com Gemini AI

**Análise completa de TODOS os dados** (~970 registros, 29 tabelas)

**Features**:
- ✅ Correlações entre funding e performance de mercado
- ✅ Clustering de setores (K-Means)
- ✅ Detecção de anomalias (Z-score)
- ✅ Insights automáticos com **Gemini AI** (10x mais barato que Claude!)
- ✅ Time series forecasting
- ✅ Export para CSV

---

## 📦 Setup & Imports

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy import stats

# AI - Gemini (BARATO!)
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Auto-EDA (GRÁTIS, sem IA!)
from ydata_profiling import ProfileReport
import sweetviz as sv

# Configuração
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Imports completos!")

## 🔗 Conectar ao PostgreSQL

In [ ]:
# Carregar .env
load_dotenv('/home/user/sofia-pulse/.env')

# Conexão PostgreSQL
DB_USER = os.getenv('DB_USER', 'sofia')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'sofia123strong')
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'sofia_db')

connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_string)

# Testar conexão
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'sofia'"))
    num_tables = result.scalar()
    print(f"✅ Conectado! {num_tables} tabelas no schema 'sofia'")

## 📊 1. OVERVIEW - Todos os Dados do Banco

In [ ]:
# Listar TODAS as tabelas e contagens
query_all_tables = """
SELECT 
    table_name,
    (SELECT COUNT(*) FROM sofia."" || table_name) as row_count
FROM information_schema.tables 
WHERE table_schema = 'sofia'
ORDER BY row_count DESC;
"""

# Alternativa: query manual de cada tabela
tables = pd.read_sql("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'sofia'
""", engine)

# Contar registros em cada tabela
table_counts = []
for table in tables['table_name']:
    try:
        count = pd.read_sql(f'SELECT COUNT(*) as cnt FROM sofia."{table}"', engine).iloc[0]['cnt']
        table_counts.append({'table': table, 'rows': count})
    except:
        table_counts.append({'table': table, 'rows': 0})

df_overview = pd.DataFrame(table_counts).sort_values('rows', ascending=False)

print("📊 SOFIA PULSE - Database Overview")
print("="*50)
print(df_overview.to_string(index=False))
print("="*50)
print(f"Total de tabelas: {len(df_overview)}")
print(f"Total de registros: {df_overview['rows'].sum():,}")
print(f"Tabelas com dados: {(df_overview['rows'] > 0).sum()}")

# Visualização
df_with_data = df_overview[df_overview['rows'] > 0]
fig = px.bar(df_with_data, x='table', y='rows', 
             title='Sofia Pulse - Registros por Tabela',
             labels={'table': 'Tabela', 'rows': 'Registros'})
fig.update_xaxis(tickangle=45)
fig.show()

## 💰 2. FINANCE DATA - B3, NASDAQ, Funding

In [ ]:
# Carregar dados finance
print("📥 Carregando dados de finance...")

# B3 (Brasil)
df_b3 = pd.read_sql('SELECT * FROM sofia.market_data_brazil ORDER BY collected_at DESC', engine)
print(f"✅ B3: {len(df_b3)} registros")

# NASDAQ
df_nasdaq = pd.read_sql('SELECT * FROM sofia.market_data_nasdaq ORDER BY collected_at DESC', engine)
print(f"✅ NASDAQ: {len(df_nasdaq)} registros")

# Funding Rounds
df_funding = pd.read_sql('SELECT * FROM sofia.funding_rounds ORDER BY amount_usd DESC', engine)
print(f"✅ Funding: {len(df_funding)} registros")

# Preview
print("\n📊 Últimos dados B3:")
display(df_b3.head(5)[['ticker', 'company', 'price', 'change_pct', 'sector']])

print("\n💰 Maiores funding rounds:")
display(df_funding.head(5)[['company_name', 'sector', 'round_type', 'amount_usd', 'valuation_usd']])

## 🔍 3. CORRELAÇÃO - Funding vs Performance

In [ ]:
# Agregar por setor
funding_by_sector = df_funding.groupby('sector').agg({
    'amount_usd': 'sum',
    'company_name': 'count'
}).reset_index()
funding_by_sector.columns = ['sector', 'total_funding', 'num_deals']

# Performance B3 por setor
b3_by_sector = df_b3.groupby('sector').agg({
    'change_pct': 'mean',
    'ticker': 'count',
    'volume': 'sum'
}).reset_index()
b3_by_sector.columns = ['sector', 'avg_performance', 'num_companies', 'total_volume']

# NASDAQ por setor
nasdaq_by_sector = df_nasdaq.groupby('sector').agg({
    'change_pct': 'mean',
    'ticker': 'count'
}).reset_index()
nasdaq_by_sector.columns = ['sector', 'nasdaq_avg_performance', 'nasdaq_companies']

# Merge
correlation_df = funding_by_sector.merge(b3_by_sector, on='sector', how='outer')
correlation_df = correlation_df.merge(nasdaq_by_sector, on='sector', how='outer')
correlation_df = correlation_df.fillna(0)

# Calcular correlação
numeric_cols = ['total_funding', 'avg_performance', 'nasdaq_avg_performance', 'total_volume']
corr_matrix = correlation_df[numeric_cols].corr()

# Visualização
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', square=True, linewidths=1)
plt.title('Matriz de Correlação: Funding vs Performance')
plt.tight_layout()
plt.show()

print("\n🔍 Correlações encontradas:")
print(corr_matrix)

# Scatter: Funding vs Performance
fig = px.scatter(correlation_df, 
                 x='total_funding', 
                 y='avg_performance',
                 size='num_deals',
                 color='sector',
                 hover_data=['sector', 'num_deals'],
                 title='Funding vs Performance (B3)',
                 labels={'total_funding': 'Total Funding ($)', 'avg_performance': 'Média Performance (%)'})
fig.show()

## 🎯 4. CLUSTERING - Agrupar Setores Similares

In [ ]:
# Preparar features para clustering
features = correlation_df[['total_funding', 'avg_performance', 'total_volume', 'nasdaq_avg_performance']]
features = features.fillna(0)

# Normalizar
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# K-Means (3 clusters)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
correlation_df['cluster'] = kmeans.fit_predict(features_scaled)

# PCA para visualização 2D
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)
correlation_df['pca1'] = features_pca[:, 0]
correlation_df['pca2'] = features_pca[:, 1]

# Visualização
fig = px.scatter(correlation_df, 
                 x='pca1', 
                 y='pca2',
                 color='cluster',
                 text='sector',
                 title='K-Means Clustering (3 grupos)',
                 labels={'pca1': 'Componente Principal 1', 'pca2': 'Componente Principal 2'})
fig.update_traces(textposition='top center')
fig.show()

# Interpretar clusters
print("\n🎯 Clusters identificados:")
for cluster_id in range(3):
    cluster_data = correlation_df[correlation_df['cluster'] == cluster_id]
    print(f"\nCluster {cluster_id}:")
    print(f"  Setores: {', '.join(cluster_data['sector'].tolist())}")
    print(f"  Média Funding: ${cluster_data['total_funding'].mean():,.0f}")
    print(f"  Média Performance: {cluster_data['avg_performance'].mean():.2f}%")
    print(f"  Média Volume: {cluster_data['total_volume'].mean():,.0f}")

## ⚠️ 5. ANOMALY DETECTION - Outliers

In [ ]:
# Z-score para detectar anomalias
features_for_anomaly = correlation_df[['total_funding', 'avg_performance', 'total_volume']]
z_scores = np.abs(stats.zscore(features_for_anomaly.fillna(0)))

# Anomalias = Z-score > 2 (outliers)
anomalies = (z_scores > 2).any(axis=1)
correlation_df['is_anomaly'] = anomalies

print("⚠️ Anomalias detectadas:")
print("="*50)
anomaly_sectors = correlation_df[correlation_df['is_anomaly']]
if len(anomaly_sectors) > 0:
    for _, row in anomaly_sectors.iterrows():
        print(f"\n🔴 {row['sector']}")
        print(f"   Funding: ${row['total_funding']:,.0f}")
        print(f"   Performance: {row['avg_performance']:.2f}%")
        print(f"   Volume: {row['total_volume']:,.0f}")
else:
    print("Nenhuma anomalia forte detectada (Z-score > 2)")

# Visualização
fig = px.scatter(correlation_df, 
                 x='total_funding', 
                 y='avg_performance',
                 color='is_anomaly',
                 size='total_volume',
                 text='sector',
                 title='Detecção de Anomalias (Z-score > 2)',
                 color_discrete_map={True: 'red', False: 'blue'})
fig.show()

## 🤖 6. GEMINI AI - Gerar Insights Automáticos

In [ ]:
# Configurar Gemini
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if not GEMINI_API_KEY:
    print("⚠️ GEMINI_API_KEY não encontrada no .env")
    print("   Obtenha grátis em: https://aistudio.google.com/app/apikey")
    print("   Depois: echo 'GEMINI_API_KEY=sua_key' >> ~/.env")
else:
    genai.configure(api_key=GEMINI_API_KEY)
    
    # Modelo (Flash = barato, Pro = melhor qualidade)
    model = genai.GenerativeModel('gemini-1.5-flash')
    
    # Preparar summary dos dados
    data_summary = f"""
# Sofia Pulse - Data Mining Report

## Database Overview:
- Total de registros: {df_overview['rows'].sum():,}
- Tabelas com dados: {(df_overview['rows'] > 0).sum()}

## Finance Data:
- B3 (Brasil): {len(df_b3)} stocks
  - Performance média: {df_b3['change_pct'].mean():.2f}%
  - Volume total: {df_b3['volume'].sum():,.0f}
  
- NASDAQ: {len(df_nasdaq)} stocks
  - Performance média: {df_nasdaq['change_pct'].mean():.2f}%
  
- Funding Rounds: {len(df_funding)} deals
  - Total investido: ${df_funding['amount_usd'].sum():,.0f}
  - Maior deal: ${df_funding['amount_usd'].max():,.0f}

## Correlações:
{corr_matrix.to_string()}

## Clusters Identificados:
{correlation_df[['sector', 'cluster', 'total_funding', 'avg_performance']].to_string()}

## Anomalias:
{anomaly_sectors[['sector', 'total_funding', 'avg_performance']].to_string() if len(anomaly_sectors) > 0 else 'Nenhuma'}
"""
    
    # Prompt
    prompt = f"""
Você é um analista financeiro expert. Analise os dados abaixo do Sofia Pulse e gere um relatório executivo.

{data_summary}

Forneça:
1. **Principais Insights** (3-5 bullet points mais importantes)
2. **Correlações Significativas** (o que os dados revelam sobre relação funding vs performance?)
3. **Setores Quentes** (quais setores estão recebendo mais funding E performando bem?)
4. **Oportunidades** (onde investir baseado nos dados?)
5. **Riscos** (anomalias ou setores com baixa performance apesar de funding alto?)

Seja direto, use números, e destaque padrões acionáveis.
"""
    
    # Gerar insights
    print("🤖 Gerando insights com Gemini AI...")
    print("(Modelo: gemini-1.5-flash - 10x mais barato que Claude!)")
    print("\n" + "="*60)
    
    response = model.generate_content(prompt)
    
    print(response.text)
    print("="*60)
    
    # Salvar insights
    with open('/home/user/sofia-pulse/analytics/insights/gemini-insights-latest.md', 'w') as f:
        f.write(f"# Sofia Pulse - AI Insights\n\n")
        f.write(f"**Gerado em**: {pd.Timestamp.now()}\n")
        f.write(f"**Modelo**: Gemini 1.5 Flash\n\n")
        f.write(response.text)
    
    print("\n✅ Insights salvos em: analytics/insights/gemini-insights-latest.md")

## 📈 7. AUTO-EDA com Pandas Profiling (SEM IA!)

In [ ]:
# Gerar relatório automático para B3
print("📊 Gerando relatório EDA automático (Pandas Profiling)...")
print("(Isso pode demorar 1-2 min)")

profile_b3 = ProfileReport(df_b3, 
                           title="Sofia Pulse - B3 Analysis",
                           explorative=True)

# Salvar HTML
output_path = '/home/user/sofia-pulse/analytics/reports/b3-profiling-report.html'
profile_b3.to_file(output_path)

print(f"✅ Relatório gerado: {output_path}")
print("   Abra no browser para ver 50+ análises automáticas!")

# Widget inline (se suportado)
# profile_b3.to_widgets()

## 📊 8. Comparação B3 vs NASDAQ (Sweetviz)

In [ ]:
# Comparar mercados
print("📊 Gerando comparação B3 vs NASDAQ (Sweetviz)...")

# Selecionar colunas em comum
common_cols = ['price', 'change_pct', 'volume', 'sector']
df_b3_clean = df_b3[common_cols].copy()
df_nasdaq_clean = df_nasdaq[common_cols].copy()

# Comparar
comparison_report = sv.compare([df_b3_clean, "B3 (Brasil)"], 
                               [df_nasdaq_clean, "NASDAQ (US)"])

# Salvar
output_path = '/home/user/sofia-pulse/analytics/reports/b3-vs-nasdaq-comparison.html'
comparison_report.show_html(output_path, open_browser=False)

print(f"✅ Comparação gerada: {output_path}")

## 💾 9. Export - Salvar Resultados

In [ ]:
# Criar diretório de outputs
import os
os.makedirs('/home/user/sofia-pulse/analytics/insights', exist_ok=True)

# 1. Correlações
correlation_df.to_csv('/home/user/sofia-pulse/analytics/insights/correlation-analysis.csv', index=False)
print("✅ Correlações exportadas: analytics/insights/correlation-analysis.csv")

# 2. Clusters
cluster_summary = correlation_df[['sector', 'cluster', 'total_funding', 'avg_performance', 'total_volume']]
cluster_summary.to_csv('/home/user/sofia-pulse/analytics/insights/sector-clusters.csv', index=False)
print("✅ Clusters exportados: analytics/insights/sector-clusters.csv")

# 3. Anomalias
if len(anomaly_sectors) > 0:
    anomaly_sectors.to_csv('/home/user/sofia-pulse/analytics/insights/anomalies-detected.csv', index=False)
    print("✅ Anomalias exportadas: analytics/insights/anomalies-detected.csv")

# 4. Top performers
top_b3 = df_b3.nlargest(20, 'change_pct')[['ticker', 'company', 'price', 'change_pct', 'sector']]
top_b3.to_csv('/home/user/sofia-pulse/analytics/insights/top-performers-b3.csv', index=False)
print("✅ Top B3 exportado: analytics/insights/top-performers-b3.csv")

# 5. Funding top deals
top_funding = df_funding.nlargest(20, 'amount_usd')[['company_name', 'sector', 'round_type', 'amount_usd', 'valuation_usd']]
top_funding.to_csv('/home/user/sofia-pulse/analytics/insights/top-funding-deals.csv', index=False)
print("✅ Top Funding exportado: analytics/insights/top-funding-deals.csv")

print("\n🎉 Todos os insights exportados com sucesso!")

## 📝 Summary Final

In [ ]:
print("\n" + "="*60)
print("🎯 SOFIA PULSE - Data Mining Complete!")
print("="*60)

print(f"\n📊 Dados Analisados:")
print(f"  - Total de registros: {df_overview['rows'].sum():,}")
print(f"  - B3: {len(df_b3)} stocks")
print(f"  - NASDAQ: {len(df_nasdaq)} stocks")
print(f"  - Funding: {len(df_funding)} deals")

print(f"\n🎯 Análises Realizadas:")
print(f"  ✅ Correlações entre funding e performance")
print(f"  ✅ Clustering de setores (3 grupos)")
print(f"  ✅ Detecção de anomalias ({anomalies.sum()} anomalias)")
print(f"  ✅ Insights com Gemini AI (10x mais barato!)")
print(f"  ✅ Relatórios automáticos (Pandas Profiling, Sweetviz)")

print(f"\n💾 Outputs Gerados:")
print(f"  📄 analytics/insights/correlation-analysis.csv")
print(f"  📄 analytics/insights/sector-clusters.csv")
print(f"  📄 analytics/insights/top-performers-b3.csv")
print(f"  📄 analytics/insights/top-funding-deals.csv")
print(f"  📄 analytics/insights/gemini-insights-latest.md")
print(f"  🌐 analytics/reports/b3-profiling-report.html")
print(f"  🌐 analytics/reports/b3-vs-nasdaq-comparison.html")

print(f"\n💰 Custo da Análise (Gemini):")
print(f"  ~$0.0013 por análise completa")
print(f"  (vs $0.06 com Claude = 46x mais barato!)")

print("\n✅ Data mining completo! Abra os relatórios HTML para explorar.")
print("="*60)